In [2]:
import cudf
import pandas as pd
from tqdm import tqdm

In [3]:
import torch

# Verificar si PyTorch detecta la GPU
print("CUDA disponible:", torch.cuda.is_available())

# Verificar el dispositivo actual
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Dispositivo actual:", device)

CUDA disponible: True
Dispositivo actual: cuda


In [4]:
df = cudf.read_csv("bitcoin_tweet_procress2.csv")
df

,user_name,user_location,user_created,user_followers,user_friends,user_favourites,user_verified,date,text,text2,hashtags,source
0,DeSota Wilson,Austria,2009-04-26,8534.0,7605.0,4838.0,False,2021-02-10,Blue Ridge Bank shares halted by NYSE after #b...,blue ridge bank shares halted by nyse after bi...,['bitcoin'],twitter web app
1,DeSota Wilson,Austria,2009-04-26,8534.0,7605.0,4838.0,False,2021-02-10,.'s #bitcoin investment is revolutionary for #...,s bitcoin investment is revolutionary for cryp...,"['bitcoin', 'crypto']",twitter web app
2,DeSota Wilson,Austria,2009-04-26,8534.0,7605.0,4838.0,False,2021-02-10,#Bitcoin institutional demand accelerates in 2...,bitcoin institutional demand accelerates in sa...,"['bitcoin', 'btc', 'crypto', 'cryptocurrency']",twitter web app
3,Humayun MHA,Pakistan,2016-08-18,1392.0,780.0,960.0,False,2021-02-10,$ADA Sky rocket 🔥🚀 #Patience 🔑$1 Soon !!! #ada...,ada sky rocket patience soon ada adausdt btc b...,"['patience', 'ada', 'adausdt', 'btc', 'bitcoin...",twitter for android
4,Stefan Gyarmati,Switzerland,2009-06-01,62.0,288.0,2656.0,False,2021-02-10,#Bitcoin #BTC #ADA #DOT Mastercard Will Let Me...,bitcoin btc ada dot mastercard will let mercha...,"['bitcoin', 'btc', 'ada', 'dot']",twitter for iphone
...,...,...,...,...,...,...,...,...,...,...,...,...
1524467,"SaitamaUniverse, CFA",United States,2020-11-01,6969.0,7201.0,19010.0,False,2023-01-06,#Saitama Gaming: #Gaming as an industry today ...,saitama gaming gaming as an industry today is ...,"['saitama', 'gaming', 'crypto', 'saitama', 'et...",twitter web app
1524468,John Morgan,Norway,2013-12-08,28480.0,621.0,112.0,False,2023-01-06,#CZ Proclaims Bianance Surge Is Due to #BTC 'Z...,cz proclaims bianance surge is due to btc zero...,"['cz', 'btc', 'trading']",twitter web app
1524469,Crypto Rosetta,United Kingdom,2022-07-11,224.0,1201.0,410.0,False,2023-01-06,Anonymous Crypto Wallet Moves $250M In Bitcoin...,anonymous crypto wallet moves m in bitcoin aft...,"['bitcoin', 'btc', 'bitcoinprice', 'recession'...",twitter for iphone
1524470,CryptoPredator,Pakistan,2022-12-31,17.0,133.0,11.0,False,2023-01-06,$DXY will fill the FVG and if PUMP from the lo...,dxy will fill the fvg and if pump from the loc...,"['bitcoin', 'btc', 'dxy', 'crypto', 'finance']",twitter for iphone


In [5]:
df = df.to_pandas()

In [6]:
import nltk
from nltk.corpus import stopwords

In [7]:
nltk.download('stopwords')
def remove_stopwords(text):
    if pd.isnull(text):
        return text
    stop_words = set(stopwords.words('english'))
    words = text.split()
    filtered_text = [word for word in words if word.lower() not in stop_words]
    return ' '.join(filtered_text)

df['text2'] = df['text2'].apply(remove_stopwords)

[nltk_data] Downloading package stopwords to /home/sante/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [8]:
from collections import defaultdict

In [9]:
def map_reduce(texts):
    word_count = defaultdict(int)
    for text in texts:
        if pd.notnull(text):
            words = text.split()
            for word in words:
                word_count[word] += 1
    sorted_word_count = dict(sorted(word_count.items(), key=lambda item: item[1], reverse=True))
    return sorted_word_count
word_frequencies = map_reduce(df['text2'])

In [10]:
len(word_frequencies)

601062

In [11]:
words_set = list(word_frequencies.items())[-599600:]

In [12]:
least_frequent_words_set = set(dict(list(word_frequencies.items())[-599600:]).keys())
def remove_least_frequent(text):
    if pd.isnull(text):
        return text
    words = text.split()
    filtered_words = [word for word in words if word not in least_frequent_words_set]
    return ' '.join(filtered_words)
df['sentences'] = df['text2'].apply(remove_least_frequent)

In [13]:
df = df.dropna()
df

,user_name,user_location,user_created,user_followers,user_friends,user_favourites,user_verified,date,text,text2,hashtags,source,sentences
0,DeSota Wilson,Austria,2009-04-26,8534.0,7605.0,4838.0,False,2021-02-10,Blue Ridge Bank shares halted by NYSE after #b...,blue ridge bank shares halted nyse bitcoin atm...,['bitcoin'],twitter web app,blue bank shares bitcoin
1,DeSota Wilson,Austria,2009-04-26,8534.0,7605.0,4838.0,False,2021-02-10,.'s #bitcoin investment is revolutionary for #...,bitcoin investment revolutionary crypto firms ...,"['bitcoin', 'crypto']",twitter web app,bitcoin investment crypto may yet
2,DeSota Wilson,Austria,2009-04-26,8534.0,7605.0,4838.0,False,2021-02-10,#Bitcoin institutional demand accelerates in 2...,bitcoin institutional demand accelerates says ...,"['bitcoin', 'btc', 'crypto', 'cryptocurrency']",twitter web app,bitcoin institutional demand says ceo btc btc ...
3,Humayun MHA,Pakistan,2016-08-18,1392.0,780.0,960.0,False,2021-02-10,$ADA Sky rocket 🔥🚀 #Patience 🔑$1 Soon !!! #ada...,ada sky rocket patience soon ada adausdt btc b...,"['patience', 'ada', 'adausdt', 'btc', 'bitcoin...",twitter for android,ada patience soon ada btc bitcoin
4,Stefan Gyarmati,Switzerland,2009-06-01,62.0,288.0,2656.0,False,2021-02-10,#Bitcoin #BTC #ADA #DOT Mastercard Will Let Me...,bitcoin btc ada dot mastercard let merchants a...,"['bitcoin', 'btc', 'ada', 'dot']",twitter for iphone,bitcoin btc ada dot let accept payments crypto...
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1524467,"SaitamaUniverse, CFA",United States,2020-11-01,6969.0,7201.0,19010.0,False,2023-01-06,#Saitama Gaming: #Gaming as an industry today ...,saitama gaming gaming industry today already l...,"['saitama', 'gaming', 'crypto', 'saitama', 'et...",twitter web app,saitama gaming gaming industry today already b...
1524468,John Morgan,Norway,2013-12-08,28480.0,621.0,112.0,False,2023-01-06,#CZ Proclaims Bianance Surge Is Due to #BTC 'Z...,cz proclaims bianance surge due btc zero tradi...,"['cz', 'btc', 'trading']",twitter web app,due btc zero trading fee
1524469,Crypto Rosetta,United Kingdom,2022-07-11,224.0,1201.0,410.0,False,2023-01-06,Anonymous Crypto Wallet Moves $250M In Bitcoin...,anonymous crypto wallet moves bitcoin lying do...,"['bitcoin', 'btc', 'bitcoinprice', 'recession'...",twitter for iphone,crypto wallet moves bitcoin btc bitcoinprice b...
1524470,CryptoPredator,Pakistan,2022-12-31,17.0,133.0,11.0,False,2023-01-06,$DXY will fill the FVG and if PUMP from the lo...,dxy fill fvg pump local support may see dump s...,"['bitcoin', 'btc', 'dxy', 'crypto', 'finance']",twitter for iphone,dxy pump local support may see dump soon going...


In [14]:
import torch
from transformers import BertTokenizer, BertForSequenceClassification
import torch.nn.functional as F

model_name = "bert-base-uncased" 
model = BertForSequenceClassification.from_pretrained(model_name)
tokenizer = BertTokenizer.from_pretrained(model_name)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("device:", device)
model.to(device)
model.eval()

/home/sante/miniconda3/envs/rapids-24.04/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


device: cuda


BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e

In [ ]:
from torch.utils.data import DataLoader, Dataset

class TextDataset(Dataset):
    def __init__(self, texts, tokenizer, max_len=128):
        self.texts = texts
        self.tokenizer = tokenizer
        self.max_len = max_len

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        text = self.texts[idx]
        encoding = self.tokenizer.encode_plus(
            text,
            add_special_tokens=True,
            max_length=self.max_len,
            padding='max_length',
            truncation=True,
            return_tensors='pt'
        )
        return {
            'input_ids': encoding['input_ids'].squeeze(0),
            'attention_mask': encoding['attention_mask'].squeeze(0)
        }

def classify_in_batches(df, text_column, model, tokenizer, device, batch_size=128):
    dataset = TextDataset(df[text_column].tolist(), tokenizer)
    dataloader = DataLoader(dataset, batch_size=batch_size, pin_memory=True)

    all_preds = []

    with torch.no_grad():
        for batch in tqdm(dataloader, desc="Clasificando batches"):
            input_ids = batch['input_ids'].to(device, non_blocking=True)
            attention_mask = batch['attention_mask'].to(device, non_blocking=True)

            with torch.cuda.amp.autocast():
                outputs = model(input_ids, attention_mask=attention_mask)
                logits = outputs.logits
                probs = F.softmax(logits, dim=1)

            preds = torch.argmax(probs, dim=1)
            all_preds.extend(preds.cpu())
    
    all_preds = torch.cat([pred.unsqueeze(0) if pred.dim() == 0 else pred for pred in all_preds]).numpy()

    df['predicted_class'] = all_preds

    return df

df = classify_in_batches(df, 'text2', model, tokenizer, device, batch_size=128)

df[['text', 'text2', 'predicted_class', 'probabilities']]


In [20]:
df

,user_name,user_location,user_created,user_followers,user_friends,user_favourites,user_verified,date,text,text2,hashtags,source,sentences,predicted_class
0,DeSota Wilson,Austria,2009-04-26,8534.0,7605.0,4838.0,False,2021-02-10,Blue Ridge Bank shares halted by NYSE after #b...,blue ridge bank shares halted nyse bitcoin atm...,['bitcoin'],twitter web app,blue bank shares bitcoin,1
1,DeSota Wilson,Austria,2009-04-26,8534.0,7605.0,4838.0,False,2021-02-10,.'s #bitcoin investment is revolutionary for #...,bitcoin investment revolutionary crypto firms ...,"['bitcoin', 'crypto']",twitter web app,bitcoin investment crypto may yet,1
2,DeSota Wilson,Austria,2009-04-26,8534.0,7605.0,4838.0,False,2021-02-10,#Bitcoin institutional demand accelerates in 2...,bitcoin institutional demand accelerates says ...,"['bitcoin', 'btc', 'crypto', 'cryptocurrency']",twitter web app,bitcoin institutional demand says ceo btc btc ...,1
3,Humayun MHA,Pakistan,2016-08-18,1392.0,780.0,960.0,False,2021-02-10,$ADA Sky rocket 🔥🚀 #Patience 🔑$1 Soon !!! #ada...,ada sky rocket patience soon ada adausdt btc b...,"['patience', 'ada', 'adausdt', 'btc', 'bitcoin...",twitter for android,ada patience soon ada btc bitcoin,1
4,Stefan Gyarmati,Switzerland,2009-06-01,62.0,288.0,2656.0,False,2021-02-10,#Bitcoin #BTC #ADA #DOT Mastercard Will Let Me...,bitcoin btc ada dot mastercard let merchants a...,"['bitcoin', 'btc', 'ada', 'dot']",twitter for iphone,bitcoin btc ada dot let accept payments crypto...,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1524467,"SaitamaUniverse, CFA",United States,2020-11-01,6969.0,7201.0,19010.0,False,2023-01-06,#Saitama Gaming: #Gaming as an industry today ...,saitama gaming gaming industry today already l...,"['saitama', 'gaming', 'crypto', 'saitama', 'et...",twitter web app,saitama gaming gaming industry today already b...,1
1524468,John Morgan,Norway,2013-12-08,28480.0,621.0,112.0,False,2023-01-06,#CZ Proclaims Bianance Surge Is Due to #BTC 'Z...,cz proclaims bianance surge due btc zero tradi...,"['cz', 'btc', 'trading']",twitter web app,due btc zero trading fee,1
1524469,Crypto Rosetta,United Kingdom,2022-07-11,224.0,1201.0,410.0,False,2023-01-06,Anonymous Crypto Wallet Moves $250M In Bitcoin...,anonymous crypto wallet moves bitcoin lying do...,"['bitcoin', 'btc', 'bitcoinprice', 'recession'...",twitter for iphone,crypto wallet moves bitcoin btc bitcoinprice b...,1
1524470,CryptoPredator,Pakistan,2022-12-31,17.0,133.0,11.0,False,2023-01-06,$DXY will fill the FVG and if PUMP from the lo...,dxy fill fvg pump local support may see dump s...,"['bitcoin', 'btc', 'dxy', 'crypto', 'finance']",twitter for iphone,dxy pump local support may see dump soon going...,1


In [17]:
df.to_csv("bitcoin_tweet_sentiment_analysis.csv", index=False)